In [ ]:
!git clone https://github.com/aravinds-kannappan/MarioGPT.git

In [ ]:
import os
import sys
import subprocess
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM